![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Calibración de parámetros de redes neuronales
En este notebook aprenderá a calibrar parámetros de redes neuronales con dos diferentes metodologías usando la librería Keras y sklearn.model_selection.

## Instrucciones Generales

Este notebook esta compuesto por tres secciones. En la primera sección, usted beberá calibrar una red neuronal para predecir el precio de una casa con el set de datos Boston Housing Data y con el método de busqueda por cuadrícula (Grid Search). En la segunda, se usará el mismo dataset pero usted beberá calibrar la red neuronal con el método de busqueda aleatoria (Randomized Search), lo que le permitirá identificar las ventajas y diferencias entre los dos métodos. Para conocer más detalles de la base, puede ingresar al siguiente [vínculo](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/).
   
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

## Importar base de datos y librerías

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
import pandas as pd
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.datasets import load_boston
# Carga de datos de la librería sklearn
boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

## Definición de variables predictoras  y de interés

In [ ]:
# Definición de variables predictoras (X)
X = boston.drop(boston.columns[-1],axis=1)
# Definición de variable de interés (y)
Y = pd.DataFrame(np.array(boston_dataset.target), columns=['labels'])

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usando la función train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X,Y, test_size=0.3 ,random_state=22)

In [ ]:
# Normalización de variables predictoras (X) con la función StandardScaler

# Definición de la función StandardScaler
scaler = StandardScaler()
scaler.fit(X)

# Transformación de los set de entrenamiento y test
X_train = pd.DataFrame(data=scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [ ]:
# Definición de dimensiones de salida (variables de interés)
output_var = Y_train.shape[1]
print(output_var, ' output variables')

In [ ]:
# Definición de dimensiones de entrada (variables predictoras)
dims = X_train.shape[1]
print(dims, 'input variables')

In [ ]:
# Separación de datos de entrenamiento para considerar un set de validación durante entrenamiento
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15, random_state=42)

## Red Neuronal

En esta sección se construirá un modelo de red neuronal mediante una función (*nn_model_params*) que parametriza los diferentes parámetros a calibrar. 

In [ ]:
# Para correr esta sección instale la librería livelossplot
pip install livelossplot

In [ ]:
# Importación librerías
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.callbacks import EarlyStopping
from keras import backend as K
from livelossplot import PlotLossesKeras

In [ ]:
# Definición de función que crea una red neuronal a partir de diferentes parámetros (nn_model_params)
# En esta función se consideran 7 parámetos a calibrar, sin embargo se pueden agregar o quitar tantos como lo consideren pertinente
def nn_model_params(optimizer ,
                    neurons,
                    batch_size,
                    epochs,
                    activation,
                    patience,
                    loss):
    
    K.clear_session()

    # Definición red neuronal con la función Sequential()
    model = Sequential()
    
    # Definición de las capas de la red con el número de neuronas y la función de activación definidos en la función nn_model_params
    model.add(Dense(neurons, input_shape=(dims,), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(output_var, activation=activation))

    # Definición de función de perdida con parámetros definidos en la función nn_model_params
    model.compile(optimizer = optimizer, loss=loss)
    
    # Definición de la función EarlyStopping con parámetro definido en la función nn_model_params
    early_stopping = EarlyStopping(monitor="val_loss", patience = patience)

    # Entrenamiento de la red neuronal con parámetros definidos en la función nn_model_params
    model.fit(X_train, Y_train,
              validation_data = (X_val, Y_val),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=[early_stopping, PlotLossesKeras()],
              verbose=True
              )
     
    return model

In [ ]:
# Definición de parámetros y sus valores sobre los que se va a calibrar
nn_params = {
    'optimizer': ['adam','sgd'],
    'activation': ['relu'],
    'batch_size': [64,128],
    'neurons':[64,256],
    'epochs':[20,50],
    'patience':[2,5],
    'loss':['mean_squared_error']
}

## Método busqueda por cuadrícula (Grid Search)

La búsqueda por cuadrícula es un método de calibración de parámetros donde se considera exhaustivamente todas las combinaciones de parámetros de un conjunto determinado.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definición de red neuronal usando el wrapper KerasRegressor y usando como argumento build_fn en la función nn_model_params
# para más detalles del wrapper puede ingresar al siguiente link https://faroit.com/keras-docs/1.2.2/scikit-learn-api/
nn_model = KerasRegressor(build_fn=nn_model_params, verbose=0)

# Definición método GridSearch para la calibración de parámetros definidos en nn_params
gs = GridSearchCV(nn_model, nn_params, cv=3)
gs.fit(X_train, Y_train)

print('Los mejores parametros segun Grid Search:', gs.best_params_)

## Método busqueda aleatoria (Randomized Search)

La busqueda aleatoria es un método de calibración de parámetros donde se consideran combinaciones aleatorias de parámetros de un conjunto determinado.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Definición de red neuronal usando el wrapper KerasRegressor y usando como argumento build_fn en la función nn_model_params
nn_model = KerasRegressor(build_fn=nn_model_params, verbose=0)

# Definición método GridSearch para la calibración de parámetros definidos en nn_params
rs = RandomizedSearchCV(nn_model, param_distributions=nn_params, n_iter=5, cv=3)
rs.fit(X_train, Y_train)

print('Los mejores parametros segun Randomnized Search:', rs.best_params_)

## Comparación de resultados
En esta última sección se comparará el desempeño de la red neuronal en el set de test con la métrica del MSE, dados los parámetros calibrados que se obtuvieron en cada uno de los métodos.

In [ ]:
# Desempeño de modelo con los parametros de Grid Search
model = nn_model_params(optimizer = 'adam',
                        neurons=256,
                        batch_size=64,
                        epochs=50,
                        activation='relu',
                        patience=5,
                        loss='mean_squared_error')
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Valor de ", model.metrics_names[0] ," o MSE dados los parámetros de Grid Search:", scores)

In [ ]:
# Desempeño de modelo con los parametros de Randomized Search
model = nn_model_params(optimizer = 'adam',
                        neurons=256,
                        batch_size=64,
                        epochs=20,
                        activation='relu',
                        patience=5,
                        loss='mean_squared_error')
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Valor de ", model.metrics_names[0] ," o MSE dados los parámetros de Randomized Search:", scores)